In [ ]:
# Import tensorflow - 
# You can use: 
# pip install tensorflow
# to install these packages
# make sure that you have seaborn version 0.11.0 installed 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
from numpy.random import normal
from tensorflow.keras.datasets.mnist import load_data
import random
random.seed(100)
plt.style.use('seaborn')
from torch.utils.data import TensorDataset

In [ ]:
#Load data 
# https://www.tensorflow.org/api_docs/python/tf/keras/datasets/mnist/load_data) 
(x_train, y_train), (x_test, y_test)=load_data(
    path='mnist.npz'
)
#ensure values are between 0 and 1
X_2 = x_train/255 #1.0/ (1 + np.exp(-1 *x_train)) 

#add noise
X_2_noise=[]

for i in range(len(X_2)):
    X_2_noise.append(X_2[i] +  np.random.normal(0, np.sqrt(0.1), X_2[i].shape))


#display the data
plt.imshow(X_2[0])
plt.show()
plt.imshow(X_2_noise[0])
plt.show()


In [ ]:
#Build auto-encoder
#reshape array
X_2_noise_reshaped= np.reshape(X_2_noise, (len(X_2_noise), 28*28))

target=np.reshape(X_2, (len(X_2_noise), 28*28))


#build model
auto_enc_11 = nn.Sequential(
    nn.Linear(28*28, 300),
    nn.Hardtanh(), #nn.Hardtanh(),
    nn.Linear(300,28*28) 
)


#define loss function
loss_fn = torch.nn.MSELoss()

#define learning rate
learning_rate = 1e-1

#convert dataset to tensors
#X_scaled = torch.from_numpy(X_scaled)

X_tensor= torch.from_numpy(X_2_noise_reshaped)
X_tensor = X_tensor.float()
target_tensor= torch.from_numpy(target)
target_tensor= target_tensor.float()

#train model
for t in range (5000):
  y_pred = auto_enc_11(X_tensor)
  loss = loss_fn(y_pred, target_tensor)
  if t % 1000 == 999:
        print(t, loss.item())
  auto_enc_11.zero_grad()
  loss.backward()
  with torch.no_grad():
        for param in auto_enc_11.parameters():
            param -= learning_rate * param.grad 

In [ ]:
#plot noised image, denoised image, and original image without noise 

X_reconstructed = auto_enc_11(X_tensor)
X_reconstructed_pic = np.reshape(X_reconstructed.detach().numpy(), (len(X_2_noise), 28, 28))
#plot the distribution of the reconstruction error
sns.distplot(target_tensor - X_reconstructed.detach().numpy())
plt.xlabel("Reconstruction Error")
plt.title("Reconstruction Error Distribution for Autoencoder")
plt.show()
#plot ditribution without denoiser 
sns.distplot(target_tensor - X_tensor)
plt.xlabel("Reconstruction Error")
plt.title("Reconstruction Error Distribution no Autoencoder")
plt.show()

#
#display the data
plt.imshow(X_2[0])
plt.show()
plt.imshow(X_2_noise[0])
plt.show()
plt.imshow(X_reconstructed_pic[0])
plt.show()